## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/79-googleq-a-train-glove-bilstm/model.h5'
tokenizer_path = '/kaggle/input/79-googleq-a-train-glove-bilstm/tokenizer.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())

# Model parameters

In [5]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 20000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 200
ANSWER_MAX_LEN = 200

## Test set

In [6]:
tokenizer = load(tokenizer_path)

# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_seq = test['question_body_uncased']
X_test_answer_seq = test['answer_uncased']

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
X_test_body_seq = tokenizer.texts_to_sequences(X_test_body_seq)
X_test_answer_seq = tokenizer.texts_to_sequences(X_test_answer_seq)

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
X_test_body_seq = pad_sequences(X_test_body_seq, maxlen=BODY_MAX_LEN)
X_test_answer_seq = pad_sequences(X_test_answer_seq, maxlen=ANSWER_MAX_LEN)

X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq]

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
    return encoder

## Load embedding

In [8]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
print('n unknown words (GloVe): ', len(glove_unknown_words))

n unknown words (GloVe):  3306


In [9]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = SpatialDropout1D(0.5)(title)
title_out = Bidirectional(LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=False))(title)

# Body sequence
input_body_seq = Input(shape=(BODY_MAX_LEN,), dtype=tf.float32, name='input_body_seq')
body = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_body_seq)
body = SpatialDropout1D(0.5)(body)
body_out = Bidirectional(LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=False))(body)

# Answer sequence
input_answer_seq = Input(shape=(ANSWER_MAX_LEN,), dtype=tf.float32, name='input_answer_seq')
answer = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_answer_seq)
answer = SpatialDropout1D(0.5)(answer)
answer_out = Bidirectional(LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=False))(answer)


# Output
x = Concatenate()([title_out, body_out, answer_out])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_initializer='lecun_normal')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='lecun_normal', name='output')(x)
model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_body_seq (InputLayer)     [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 300)      17424600    input_title_seq[0][0]            
______________________________________________________________________________________________

# Make predictions

In [10]:
Y_test = model.predict(X_test)

In [11]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.900090,0.680271,0.115659,0.737549,0.787213,0.797243,0.616816,0.573640,0.317028,...,0.863370,0.890296,0.623411,0.933481,0.948587,0.817364,0.151297,0.068487,0.726539,0.893723
1,46,0.898742,0.550148,0.001597,0.724365,0.786937,0.916011,0.540955,0.451592,0.073057,...,0.735049,0.948116,0.639083,0.973760,0.983507,0.880854,0.942167,0.128643,0.071181,0.887187
2,70,0.941117,0.788593,0.045537,0.840196,0.921774,0.911750,0.651458,0.627320,0.210760,...,0.908931,0.926994,0.666162,0.962172,0.969713,0.871347,0.023097,0.025031,0.828997,0.927612
3,132,0.877245,0.498470,0.000847,0.704025,0.783040,0.963653,0.532037,0.432362,0.126490,...,0.707911,0.953301,0.676072,0.978431,0.991200,0.890807,0.908195,0.151757,0.434379,0.895229
4,200,0.892686,0.531383,0.027163,0.781490,0.783609,0.849363,0.594145,0.527994,0.239293,...,0.738182,0.910018,0.635445,0.956324,0.962782,0.830265,0.308456,0.119889,0.638091,0.890864


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.878018,0.582001,0.037478,0.696725,0.788886,0.841588,0.570911,0.487978,0.219218,...,0.773319,0.925388,0.650108,0.960299,0.969209,0.852006,0.574623,0.149017,0.486674,0.896407
std,2812.670060,0.045444,0.121149,0.061205,0.094152,0.084635,0.122765,0.041776,0.072925,0.116894,...,0.089554,0.025421,0.034400,0.015320,0.015396,0.035860,0.309259,0.064925,0.248183,0.022631
min,39.000000,0.732810,0.333994,0.000084,0.334060,0.399735,0.280585,0.490684,0.371670,0.026536,...,0.577328,0.816425,0.553006,0.900960,0.907850,0.744905,0.005558,0.013092,0.067307,0.785125
25%,2572.000000,0.847013,0.480708,0.003397,0.654108,0.745639,0.796731,0.539679,0.437531,0.122006,...,0.698920,0.910222,0.629742,0.952945,0.959589,0.828738,0.317071,0.113614,0.271256,0.882613
50%,5093.000000,0.880727,0.571174,0.009732,0.711216,0.793726,0.886916,0.562662,0.465658,0.202544,...,0.774989,0.928905,0.641470,0.963318,0.972627,0.854470,0.665771,0.146350,0.480348,0.897038
75%,7482.000000,0.915709,0.693448,0.042027,0.759074,0.838388,0.922810,0.600550,0.528589,0.297842,...,0.857338,0.942436,0.661985,0.970674,0.980460,0.878156,0.850494,0.185384,0.683283,0.912149
max,9640.000000,0.972173,0.837683,0.436324,0.876653,0.979276,0.981968,0.705219,0.758570,0.591652,...,0.948980,0.976386,0.759132,0.989381,0.995778,0.936282,0.972382,0.374279,0.980950,0.953783
